# Preprocessing

In [1]:
import json
import uuid

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from difflib import SequenceMatcher
from IPython.display import HTML

from utils.code_utils import tokenize_python_code

pd.set_option('display.max_columns', None)

codenet_root = 'datasets/codenetpy/'

In [2]:
with open(codenet_root + 'codenetpy_train.json', 'r') as f:
    data = json.load(f)["data"]
    
with open(codenet_root + 'codenetpy_test.json', 'r') as f:
    test_data = json.load(f)["data"]
    

train_df = pd.DataFrame(data)
test_df = pd.DataFrame(test_data)

codenetpy_df = pd.DataFrame(test_data)

display(codenetpy_df)
display(codenetpy_df.info())

,original_src,changed_src,problem_id,original_id,changed_id,language,filename_ext,original_status,returncode,error_class,error_class_extra,error,output
0,"n = input()[::-1]\ndp = [[0, 0] for i in range...","n = input()[::-1]\ndp = [[0, 0] for i in range...",p02775,s756042102,s504568034,Python,py,Runtime Error,0,0,,,8\n
1,"s,t=input().split()\na,b=map(int,input().split...","s,t=input().split()\na,b=map(int,input().split...",p02777,s656012100,s321794177,Python,py,Runtime Error,1,SyntaxError,SyntaxError: invalid syntax,"File ""/home/alex/Documents/research/bug-dete...",
2,"S,T = input().split()\nA,B = int(input().split...","S,T = input().split()\nA,B = map(int,input().s...",p02777,s221552735,s406950594,Python,py,Runtime Error,1,TypeError,"TypeError: int() argument must be a string, a ...","Traceback (most recent call last):\n File ""/h...",
3,"S,T=map(input().split())\nA,B=map(int,input()....","S,T=map(str,input().split())\nA,B=map(int,inpu...",p02777,s852594157,s086339148,Python,py,Runtime Error,1,TypeError,TypeError: map() must have at least two argume...,"Traceback (most recent call last):\n File ""/h...",
4,"S, T = input().split()\nA, B = input().split()...","S, T = input().split()\nA, B = map(int, input(...",p02777,s104441064,s821249435,Python,py,Runtime Error,1,TypeError,TypeError: unsupported operand type(s) for -: ...,"Traceback (most recent call last):\n File ""/h...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10906,"N = int(input())\nL = list(map(int, input().sp...","N = int(input())\nL = list(map(int, input().sp...",p04047,s420396881,s036877571,Python,py,Runtime Error,1,IndexError,IndexError: list index out of range,"Traceback (most recent call last):\n File ""/h...",
10907,\nnum = int(input())\nli = int(input().split()...,n = int(input())\nli = [int(x) for x in input(...,p04047,s838662080,s028090472,Python,py,Runtime Error,1,TypeError,"TypeError: int() argument must be a string, a ...","Traceback (most recent call last):\n File ""/h...",
10908,from collections import Counter\n \nn = int(in...,"n = int(input())\n \narr = list(map(int, input...",p04047,s112597968,s888016571,Python,py,Runtime Error,1,TypeError,TypeError: unsupported operand type(s) for +=:...,"Traceback (most recent call last):\n File ""/h...",
10909,"N,X = input().split()\nN,X = int(N), int(X)\na...","N,X = input().split()\nN,X = int(N), int(X)\na...",p04048,s311100241,s323769427,Python,py,Runtime Error,1,TabError,TabError: inconsistent use of tabs and spaces ...,"File ""/home/alex/Documents/research/bug-dete...",


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10911 entries, 0 to 10910
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   original_src       10911 non-null  object
 1   changed_src        10911 non-null  object
 2   problem_id         10911 non-null  object
 3   original_id        10911 non-null  object
 4   changed_id         10911 non-null  object
 5   language           10911 non-null  object
 6   filename_ext       10911 non-null  object
 7   original_status    10911 non-null  object
 8   returncode         10911 non-null  int64 
 9   error_class        10911 non-null  object
 10  error_class_extra  10911 non-null  object
 11  error              10911 non-null  object
 12  output             10911 non-null  object
dtypes: int64(1), object(12)
memory usage: 1.1+ MB


None

In [3]:
codenetpy_df['error_class'].value_counts()

0                      5828
SyntaxError            1246
NameError              1146
TypeError               980
ValueError              632
IndexError              274
AttributeError          207
IndentationError        156
EOFError                138
TLEError                 87
ModuleNotFoundError      74
TabError                 35
ImportError              27
ZeroDivisionError        22
1                        17
SyntaxWarning             9
DeprecationWarning        8
KeyError                  7
UnboundLocalError         6
FileNotFoundError         5
-11                       2
OverflowError             2
OSError                   2
RecursionError            1
Name: error_class, dtype: int64

In [4]:
codenetpy_df['returncode'].value_counts()

 0     5837
 1     4985
-9       87
-11       2
Name: returncode, dtype: int64

In [5]:
# There are cases where the return code is 0 but the error_class is a Warning

codenetpy_df[(codenetpy_df['returncode'] == 0) & (codenetpy_df['error_class'] != '0')]['error_class'].value_counts()

DeprecationWarning    7
SyntaxWarning         2
Name: error_class, dtype: int64

In [6]:
# Most of the '0' error_class buggy submissions are TLE errors, which means they are inefficient implementations

codenetpy_df[codenetpy_df['error_class'] == '0']['original_status'].value_counts()

Time Limit Exceeded      3341
Runtime Error            2474
Memory Limit Exceeded      13
Name: original_status, dtype: int64

In [7]:
# Idealy the model will take as input the original_src code and will identify both the error_description and the location of the bug
# We can achieve this by comparing the original_src with the changed_src, which will give us the location of the bug (e.g. the modification)
# Next we will want to create a new model that will take as input the original_src, and the found bug and will learn to make the modifications
# on the original code such that it gets accepted
buggy_df = codenetpy_df[codenetpy_df['returncode'] != 0]
accepted_df = codenetpy_df[codenetpy_df['returncode'] == 0]

# renombrar columnas
buggy_df.rename(columns={'original_src': 'code_string', 'error_class_extra': 'error_obj'}, inplace=True)
accepted_df.rename(columns={'changed_src': 'code_string'}, inplace=True)

# filtrado de columnas
buggy_df = buggy_df[['code_string', 'error_obj', 'original_id']]
accepted_df = accepted_df[['code_string', 'original_id']]

# generacion de uuids
#buggy_df['uuid'] = [uuid.uuid4().hex for _ in range(len(buggy_df.index))]
buggy_df = buggy_df.set_index('original_id')
#accepted_df['uuid'] = [uuid.uuid4().hex for _ in range(len(accepted_df.index))]
accepted_df = accepted_df.set_index('original_id')
#buggy_df


/Users/angelchavez/opt/miniconda3/envs/BIFI/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
buggy_df.to_json(r'preprocess/raw.bad.json', orient='index', indent=2)
accepted_df.to_json(r'preprocess/raw.good.json', orient='index', indent=2)

# Tokenizing

In [9]:
raw_bad_df = pd.read_json('preprocess/raw.bad.json', orient='index')
raw_good_df = pd.read_json('preprocess/raw.good.json', orient='index')
#raw_df['code_toks_joined'], raw_df['anonymize_dict'] = zip(*raw_df['code_string'].map(tokenize_python_code))
display(raw_bad_df.isnull().sum())
display(raw_good_df.isnull().sum())


code_string    0
error_obj      0
dtype: int64

code_string    0
dtype: int64

In [10]:
raw_bad_df[['code_toks_joined', 'anonymize_dict']] = raw_bad_df.apply(lambda row: tokenize_python_code(row.name, row['code_string']), axis='columns', result_type='expand')
raw_bad_df['code_toks_joined'] = raw_bad_df['code_toks_joined'].str.join(" ")
display(raw_bad_df.isnull().sum())

raw_good_df[['code_toks_joined', 'anonymize_dict']] = raw_good_df.apply(lambda row: tokenize_python_code(row.name, row['code_string']), axis='columns', result_type='expand')
raw_good_df['code_toks_joined'] = raw_good_df['code_toks_joined'].str.join(" ")
display(raw_good_df.isnull().sum())


s898306249
s500983755
s987818486
s318246832
s124597664
s600648327
s218704951
s602268146
s186361785
s842975634
s050398664
s450619438
s568299280
s091760908
s512039163
s889399252
s336486941
s068982670
s884416703
s798116675
s508819528
s476914155
s119737285
s427656383
s857659942
s797876002
s642363921
s972342492
s173622424
s569254146
s738259797
s596298006
s414713006
s766948303
s728196003
s726574973
s244161751
s538766559
s922079902
s577166048
s826904643
s057155087
s782710630
s583360264
s150597558
s272256854
s850053662
s459846595
s166842299
s189147995
s121912320
s329327777
s909503008
s567499635
s979520824
s772857863
s120445089
s678141019
s841680686
s331575266
s684709730
s757693759
s271009009
s204848476
s655020173
s226190022
s961581013
s294675023
s784395989
s367645894
s634068126
s018745501
s618801681
s933575518
s718478939
s521808195
s352553747
s395003242
s322937244
s035487084
s921913108
s567020473
s847704143
s513013833
s634976152
s636273076
s851280803
s388445803
s403139727
s734769666
s835892815

code_string          0
error_obj            0
code_toks_joined    91
anonymize_dict       0
dtype: int64

code_string         0
code_toks_joined    0
anonymize_dict      0
dtype: int64

In [11]:
#raw_df.replace(np.nan,'',regex=True)
#raw_df.isnull().sum()
#raw_df_non_null = raw_df.fillna("")
raw_filtered_bad_df = raw_bad_df.dropna(how='any',axis=0) 
raw_filtered_good_df = raw_good_df.dropna(how='any',axis=0) 
display(raw_filtered_bad_df.head())
display(raw_filtered_good_df.head())
#raw_df.head()

,code_string,error_obj,code_toks_joined,anonymize_dict
s656012100,"s,t=input().split()\na,b=map(int,input().split...",SyntaxError: invalid syntax,"s , t = input ( ) . split ( ) <NEWLINE> a , b ...",{}
s221552735,"S,T = input().split()\nA,B = int(input().split...","TypeError: int() argument must be a string, a ...","S , T = input ( ) . split ( ) <NEWLINE> A , B ...",{}
s852594157,"S,T=map(input().split())\nA,B=map(int,input()....",TypeError: map() must have at least two argume...,"S , T = map ( input ( ) . split ( ) ) <NEWLINE...",{}
s104441064,"S, T = input().split()\nA, B = input().split()...",TypeError: unsupported operand type(s) for -: ...,"S , T = input ( ) . split ( ) <NEWLINE> A , B ...",{}
s628408067,"S,T = input (),split ()\nA,B = map (int, input...",NameError: name 'split' is not defined,"S , T = input ( ) , split ( ) <NEWLINE> A , B ...",{}


,code_string,code_toks_joined,anonymize_dict
s000214790,"import sys, os, math, bisect, itertools, colle...","import sys , os , math , bisect , itertools , ...",{'<COMMENT>': ['# from scipy.sparse.csgraph im...
s000341778,import math\nX = int(input())\nruto = math.cei...,import math <NEWLINE> X = int ( input ( ) ) <N...,{}
s000372326,"A,B,C,D,E,F=map(int,input().split(' '))\nwater...","A , B , C , D , E , F = map ( int , input ( ) ...",{'<STRING>': ['' '']}
s000533470,"import sys\nsys.setrecursionlimit(10**6)\n\nn,...",import sys <NEWLINE> sys . setrecursionlimit (...,"{'<COMMENT>': ['# print(i,l)']}"
s000609608,import bisect\n\nn = int(input())\nA = sorted(...,import bisect <NEWLINE> <NL> n = int ( input (...,{}


In [12]:
raw_filtered_bad_df.to_json(r'preprocess/tokenized.bad.json', orient='index', indent=2)
raw_filtered_good_df.to_json(r'preprocess/tokenized.good.json', orient='index', indent=2)


## Tratamiento adicional de JSON

In [13]:
with open('preprocess/tokenized.bad.json', 'r') as f1:
    data_bad = json.load(f1)


In [14]:
for key, obj in data_bad.items():
    for attribute, value in obj.items():
        if attribute == 'error_obj':
            new_obj = { 'msg': value }
            obj[attribute] = new_obj
            
# Serializing json
json_object_bad = json.dumps(data_bad, indent=2)
# Writing to final.bad.json.json
with open("preprocess/orig_bad_code/final.bad.json", "w") as outfile1:
    outfile1.write(json_object_bad)


In [15]:
with open('preprocess/tokenized.good.json', 'r') as f2:
    data_good = json.load(f2)

json_object_good = json.dumps(data_good, indent=2)

with open("preprocess/orig_good_code/final.good.json", "w") as outfile2:
    outfile2.write(json_object_good)